# Classify avian sequences into domestic and wild

August 18, 2020

To start on this avian flu project, I would like to use the H5N1 sequences that I have already curated for the avian-flu nextstrain build. While some of the sequences on that site are derived from the Influenza Research Database (which pulls from Genbank), others come from gisaid. Gisaid has an annotation feature where avian sequences can be annotated as domestic or wild, which is really helpful. However, most sequences are not annotated. I figured out when writing the application that sequences can generally be classified by reading the associated abstract or paper. To do that, I need to match gisaid ids with genbank ids, and use those to pull abstracts. In this notebook, I will do the following: 

1. I downloaded all available H5N1 sequences from gisaid. I did not stipulate whether they required any given segment or any minimum segment length. This downloaded metadata file should provide all the available metadata for all gisaid h5n1 segments. 
2. Read in this gisaid metadata file to a dictionary containing for each accession number, all of the associated metadata. 
3. Next, I will read in all of the fasta files for H5N1 that are currently used for the avian-flu repo. These fasta files live in `avian-flu/data/`. I have already done a bit of work to classify hosts for the avian flu build as bird, human, nonhuman mammal, and environment. I feel pretty confident that the majority of sequences that are ambiguous will be goose and duck. So, for any sequence annotated as bird whose host in the strain name is goose or duck, then I will pull out the accession number. If the accession is a gisaid accession, then I will check the metadata dictionary to determine whether a. there is a domestic/wild annotation and b. whether there is an associated genbank id.
4. If there is no annotation and no genbank id, then it will be really challenging to classify. If there is an annotation, then we will use that. If there is no annotation, but there is a genbank id, then I will record that genbank id and use it to look up and output an associated abstract for that sequence. 

In [1]:
# using the Entrez feature from bio
from Bio import Entrez
from Bio import SeqIO
import re
import time, datetime

In [2]:
def read_in_metadata_file(file):
    count = 0
    metadata_dict = {}
    genes = {"PB2":{"gisaid_id":1,"genbank_id":53},
            "PB1":{"gisaid_id":2,"genbank_id":54},
            "PA":{"gisaid_id":3,"genbank_id":55},
            "HA":{"gisaid_id":4,"genbank_id":56},
            "NP":{"gisaid_id":5,"genbank_id":57},
            "NA":{"gisaid_id":6,"genbank_id":58},
            "MP":{"gisaid_id":7,"genbank_id":59},
            "NS":{"gisaid_id":8,"genbank_id":60},}
    
    with open(file, "r") as infile:
        for line in infile:
            count += 1
            if len(line.split("\t")) < 63:
                print(count)
            strain_name = line.split("\t")[11].strip()
            passage_history = line.split("\t")[14].strip()
            host = line.split("\t")[16].strip()
            vacc_status = line.split("\t")[47].strip()
            domestic_status = line.split("\t")[51].strip()

            for gene in genes: 
                gisaid_id_column = genes[gene]['gisaid_id']
                gisaid_id = line.split("\t")[gisaid_id_column].split("|")[0]
                genbank_id_column = genes[gene]['genbank_id']
                genbank_id = line.split("\t")[genbank_id_column]

                # add to dictionary 
                metadata_dict[gisaid_id] = {"genbank_id":genbank_id,"strain":strain_name,"passage":passage_history,
                                           "vaccination_status":vacc_status,"domestic_wild":domestic_status,"host":host}

        return(metadata_dict)

In [3]:
metadata_file = "/Users/lmoncla/src/h5n1-host-classification/gisaid-data/gisaid-all-h5n1-2020-08-18.txt"
metadata_dict = read_in_metadata_file(metadata_file)

I think that I need the following functions, in this order: 
1. read in fasta file or metadata file, and classify host species in the strain name by wild or domestic if possible. 
2. I am going to need a species synonyms file to classify groups that are essentially the same
2. if duck or goose, then check to see whether that id is in the metadata dictionary and if it has a genbank id and domestic wild status
3. if that sequence has a domestic/wild status annotation, record that and move on 
4. if it does not have an annotation but does have a genbank id, then use that id to look up the corresponding manuscript and return the abstract. Record the pubmed id in the dictionary. 
5. we will require for this project to have either an unambiguous species name, an annotation, or a pubmed id that can be used to determine whether the sequence is domestic or wild. 
6. This information will all then end up getting added into the metadata file as extra columns
7. For the how annotation was assigned column, I am imagining that this will be something like, "was in list of known wild bird species", "abstract curation", or "annotated in gisaid"

sample dictionary: 
strain name :{accession: accession, genbank_id: genbank_id, domestic_wild_annotation: domestic_wild_annotation, 
PMID: pubmed_id, how_annotation_assigned: method}

In [4]:
"""I have an avian species synonyms document which both corrects spelling errors and standardizes avian host
annotations so that they all match. This document also contains assignments for these species as to whether they
are domestic or wild. Some species like duck and goose are not specific enough, but others like heron are. This
function will read in that file and output a dictionary with the original species name, the corrected name, and the 
domestic/wild classification"""

def read_in_avian_species_synonyms(synonyms_doc):
    fixes_dict = {}
    with open(synonyms_doc, "r") as infile: 
        for line in infile: 
            if not line.startswith("#"):
                if len(line.split("\t")) == 3:
                    species = line.split("\t")[0]
                    fix = line.split("\t")[1]
                    domestic_wild = line.split("\t")[2].strip()

                    fixes_dict[species] = {"fix":fix, "domestic_wild":domestic_wild}
                else:
                    species = line.split("\t")[0].strip()
                    fixes_dict[species] = {"fix":"", "domestic_wild":""}
    return(fixes_dict)

In [5]:
"""some strains have weird passage histories or should otherwise be excluded. This is a list to hold those"""

def read_in_exclude_file(exclude_file):
    exclude_dict = {}
    
    with open(exclude_file, "r") as infile: 
        for line in infile: 
            if "#" not in line and line != "\n":
                strain_name = line.split("\t")[0]
                accession = line.split("\t")[1]
                exclude_dict[strain_name] = accession
                
    return(exclude_dict)

In [6]:
"""some strains are improperly formatted and need to have a host annotation added explicitly"""

def read_in_strain_host_fixes(strain_host_fixes_file):
    
    strain_host_fixes_dict = {}
    
    with open(strain_host_fixes_file, "r") as infile: 
        for line in infile: 
            if "#" not in line:
                strain_name = line.split("\t")[0]
                host_annotation = line.split("\t")[1]
                corrected_strain = line.split("\t")[2]
                strain_host_fixes_dict[strain_name] = {"host_species":host_annotation,"corrected_strain":corrected_strain}
                
    return(strain_host_fixes_dict)

In [7]:
"""given the species fixes in the annotations file, standardize the species annotation"""

def standardize_host_species(strain_name, host_group, fixes_dict):
    
    # standardize host species names using fix file and and categorize domestic/wild by species if possible
    if host_group.lower() == "avian": 
        host_species = strain_name.split("/")[1].lower()
            
        if host_species in fixes_dict:
            species_fix = fixes_dict[host_species]['fix']
            
        else:
            print("not in dictionary: ",host_species, strain_name)
            species_fix = host_species
                
    else:
        host_species = host_group
        species_fix = host_group

    return(host_species, species_fix)

In [8]:
"""given a host species, determine whether that species has a clear domestic/wild classification"""

def check_for_annotation_by_species(host_species,fixes_dict):
    
    if host_species in fixes_dict:
        domestic_wild_status = fixes_dict[host_species]['domestic_wild']
    
    else:
        domestic_wild_status = ""
    
    return(domestic_wild_status)

In [9]:
"""given an accession, check to see whether it is from gisaid and if so, whether it has a domestic wild annotation"""

def check_for_annotation_by_gisaid(accession,metadata_dict):
    
    if accession.startswith("EPI") and len(accession) == 10 and accession in metadata_dict:
        domestic_wild_status = metadata_dict[accession]['domestic_wild'].lower()
    else:
        domestic_wild_status = ""
        
    return(domestic_wild_status)

In [10]:
def return_status_and_annotation_method(domestic_wild_status_by_species,domestic_wild_status_by_gisaid):
    
    # first check to make sure that the different methods did not return conflicting annotations
    if domestic_wild_status_by_species != "" and domestic_wild_status_by_gisaid != "" and domestic_wild_status_by_species != domestic_wild_status_by_gisaid:
        print("conflicting annotations ",domestic_wild_status_by_species,domestic_wild_status_by_gisaid)
    
    # if the species is informative and gisaid had an annotation
    elif domestic_wild_status_by_species != "" and domestic_wild_status_by_gisaid != "" and domestic_wild_status_by_species == domestic_wild_status_by_gisaid:
        domestic_wild = domestic_wild_status_by_species
        annotation_method = "species name and gisaid"
    
    # if only the species could be used the classify
    elif domestic_wild_status_by_species != "" and domestic_wild_status_by_gisaid == "":
        annotation_method = "species name"
        domestic_wild = domestic_wild_status_by_species
    
    # if only gisaid could be used to classify
    elif domestic_wild_status_by_species == "" and domestic_wild_status_by_gisaid != "":
        annotation_method = "gisaid annotation"
        domestic_wild = domestic_wild_status_by_gisaid
        
    # if neither could be used to classify
    elif domestic_wild_status_by_species == "" and domestic_wild_status_by_gisaid == "":
        domestic_wild = ""
        annotation_method = ""
    
    # if there is somehow another option I'm not thinking of
    else:
        print("something weird is happening!", domestic_wild_status_by_species,domestic_wild_status_by_gisaid)
        
    return(domestic_wild, annotation_method)

In [11]:
"""given an accession number, determine if it is from gisaid or genbank. If from gisaid, look up the corresponding
genbank id for it if available. If not, return a blank string"""

def return_genbank_accession(accession,metadata_dict):
    
    accession_stripped = accession.replace("_","").replace("-","")
 
    # if the accession is a gisaid accession
    if accession_stripped.startswith("EPI"):
        if accession_stripped in metadata_dict:
            genbank_id = metadata_dict[accession_stripped]['genbank_id']
        else: 
            genbank_id = ""
            
    elif len(accession_stripped) == 8: 
        genbank_id = accession   #for a genbank lookup, we need to retain the _ and -, but for gisaid, we do not
    
    else:
        print("this does not follow the rules ", accession)
        
    return(genbank_id)

In [12]:
"""given an accession number, pull the genbank entry that it corresponds to"""

def fetch_genbank_entry(accession_number):
    
    if accession_number != "":
        Entrez.email = "lhmoncla@gmail.com"      # need to enter an email

        # fetch from the nucleotide database, return a genbank object in text format, with id = genbank accession
        handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=accession_number)
        genbank_result = handle.read()
        
    else: 
        genbank_result = ""
        
    return(genbank_result)

In [13]:
"""given a genbank result, pull the pubmed id from it"""

def return_PMID(genbank_result):
    if genbank_result != "":
        SearchStr = 'PUBMED\\ \\ \\ [0-9]{8}'
        result = re.search(SearchStr,genbank_result)

        if result:
            PMID = result.group(0).split("   ")[1]
        else:
            PMID = ""
            
    else:
        PMID = ""
    
    return(PMID)

In [14]:
def query_genbank(genbank_id):
    from urllib.error import HTTPError

    flag = True
    
    while flag:
        try:
            time.sleep(0.35) # Sleep; genbank will throw an error if you are hitting it with more than 3 hits per second
            genbank_entry = fetch_genbank_entry(genbank_id)
            pubmed_id = return_PMID(genbank_entry)
            flag = False
        except HTTPError as e:
            time.sleep(1)
            print(e)
    
    return(pubmed_id)

In [15]:
def fix_host_species_and_classify(metadata_file, metadata_dict,fixes_dict,exclude_dict,strain_host_fixes_dict,output_file,master_dict):
    
    with open(output_file, "w") as outfile:
        line_to_write = "\t".join(["strain","virus","isolate_id","date","region","country","division","location","host","originating_lab","submitting_lab","host_species","host_species_standardized","domestic_wild","annotation_method","genbank_id","pubmed_id"])
        outfile.write(line_to_write + "\n")
    output_dict = {}
    
    with open(metadata_file, "r") as infile: 
        for line in infile:
            if "originating_lab" not in line:
                strain_name = line.split("\t")[0]
                host_group =  line.split("\t")[8].lower()
                accession = line.split("\t")[2]

                """filter out strains that should be excluded"""
                if strain_name not in exclude_dict:
                    
                    """first, see if we have already classified this strain"""
                    if strain_name in master_dict: 
                        host_species = master_dict[strain_name]['host_species']
                        host_species_standardized = master_dict[strain_name]['standardized_host_species']
                        domestic_wild = master_dict[strain_name]['domestic_wild']
                        annotation_method = master_dict[strain_name]['annotation_method']
                        host_group = master_dict[strain_name]['host_group']
                        pubmed_id = master_dict[strain_name]['pubmed_id']
                        
                        # for these, we still need to look up a new genbank id
                        genbank_id = return_genbank_accession(accession,metadata_dict)
                        
                    else:                 
                        """if the strain is in the strain host fixes doc, add in the host species here; 
                        else, standardize the host species name with the species synonyms"""
                        if strain_name in strain_host_fixes_dict:
                            host_species = strain_host_fixes_dict[strain_name]['host_species']
                            host_species_standardized = strain_host_fixes_dict[strain_name]['host_species']

                        else:
                            host_species,host_species_standardized = standardize_host_species(strain_name, host_group, fixes_dict)


                        """now, if the host group is avian, check for domestic wild status by its species name and by gisaid
                        annotation"""
                        if host_group == "avian":
                            domestic_wild_status_by_species = check_for_annotation_by_species(host_species,fixes_dict)
                            domestic_wild_status_by_gisaid = check_for_annotation_by_gisaid(accession,metadata_dict)
                            domestic_wild,annotation_method = return_status_and_annotation_method(domestic_wild_status_by_species,domestic_wild_status_by_gisaid)                

                        else:
                            domestic_wild = host_group
                            annotation_method = ""

                        """now, return the genbank ids"""
                        genbank_id = return_genbank_accession(accession,metadata_dict)


                        """pull the genbank entry and pubmed id for the associated paper"""
                        pubmed_id = query_genbank(genbank_id)

                    
                    
                    """now write out a dictionary"""
                    output_dict[strain_name] = {"host_species":host_species,"standardized_host_species":host_species_standardized,
                                               "domestic_wild":domestic_wild,"annotation_method":annotation_method,
                                                "genbank_id":genbank_id, "host_group":host_group,"pubmed_id":pubmed_id}

                    line_to_write = "\t".join([host_species,host_species_standardized,domestic_wild,annotation_method,genbank_id,pubmed_id])
                    with open(output_file, "a") as outfile: 
                        outfile.write(line.strip() + "\t" + line_to_write + "\n")
            
    return(output_dict)

In [16]:
avian_synonyms_file = "/Users/lmoncla/src/h5n1-host-classification/all-avian-species.txt"
strain_host_fixes = "/Users/lmoncla/src/h5n1-host-classification/strain-host-fixes.tsv"
exclude_file = "/Users/lmoncla/src/h5n1-host-classification/exclude.txt"

todays_date = (datetime.datetime.now()).strftime('%Y-%m-%d')
output_file_directory = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/"

avian_fixes_dict = read_in_avian_species_synonyms(avian_synonyms_file)
to_exclude_dict = read_in_exclude_file(exclude_file)
strain_host_fixes_dict = read_in_strain_host_fixes(strain_host_fixes)

In [17]:
# run for all genes to make sure I have all the hosts I need
fasta_directory = "/Users/lmoncla/src/avian-flu/data/"
metadata_directory = "/Users/lmoncla/src/avian-flu/results/"
genes_to_run = ['pb2','pb1','pa','ha','np','na','mp','ns']
all_genes_dict = {}

for gene in genes_to_run:
    fasta_file = fasta_directory + "h5n1_"+gene+".fasta"
    metadata_file = metadata_directory + "metadata_h5n1_" + gene + ".tsv"
    output_file = output_file_directory + "metadata_h5n1_"+gene+"_with_accessions_"+todays_date+".txt"
    gene_dict = fix_host_species_and_classify(metadata_file, metadata_dict,avian_fixes_dict,to_exclude_dict,strain_host_fixes_dict,output_file,all_genes_dict)
    
    # update the dictionary with new entries. This will retain the old keys and values, but add in new ones
    all_genes_dict.update(gene_dict)

In [18]:
# so, there are 9206 unique strains in our possession
len(all_genes_dict)

9206

## Pull abstracts to read

Given this new metadata file, for all bird sequences that don't have an annotation yet and do have a pubmed id, pull the abstract so that I can read it to classify. 

In [23]:
def collate_unique_PMIDs(metadata_file, PMIDs, cannot_classify, classify_by_abstract):
    
    with open(metadata_file, "r") as infile: 
        for line in infile: 
            if "originating_lab" not in line:
                strain = line.split("\t")[0]
                host_group = line.split("\t")[8].lower()
                domestic_wild = line.split("\t")[13]
                PMID = line.split("\t")[16].strip()
                
                if host_group == "avian" and domestic_wild == "" and PMID != "":
                    PMIDs.append(PMID)
                    classify_by_abstract.append(strain)
                elif host_group == "avian" and domestic_wild == "" and PMID == "":
                    cannot_classify.append(strain)
    
    PMIDs = list(set(PMIDs))
    cannot_classify = list(set(cannot_classify))
    classify_by_abstract = list(set(classify_by_abstract))
    return(PMIDs, cannot_classify, classify_by_abstract)

In [24]:
def pull_abstract(PMID):
    
    pmid_handle = Entrez.efetch(db='pubmed', id=PMID, retmode='text', rettype='abstract')
    abstract = pmid_handle.read()
    
    return(abstract)

In [25]:
def return_necessary_abstracts(PMIDs, abstracts_output_filename):
    
    with open(abstracts_output_filename, "w") as outfile: 
        outfile.write("")
    
    for p in PMIDs: 
        abstract = pull_abstract(p)
                    
        with open(abstracts_output_filename, "a") as outfile: 
            outfile.write(p + "\n")
            outfile.write(abstract + "\n")

In [27]:
metadata_directory = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/"
abstracts_output_filename = "/Users/lmoncla/src/h5n1-host-classification/abstracts/abstracts-"+todays_date+".txt"

genes_to_run = ['pb2','pb1','pa','ha','np','na','mp','ns']
unique_PMIDs = []
cannot_classify = []
classify_by_abstract = []

for gene in genes_to_run: 
    metadata_input_file = metadata_directory + "metadata_h5n1_" + gene + "_with_accessions_2020-09-28.txt"
    unique_PMIDs,cannot_classify,classify_by_abstract = collate_unique_PMIDs(metadata_input_file, unique_PMIDs, cannot_classify, classify_by_abstract)

# print the number of abstracts I need to read, and the number of strains that can't be classified at all, and the 
# number of strains that will be captured by abstract classification
print(len(unique_PMIDs), len(cannot_classify), len(classify_by_abstract))

146 1824 1292


In [142]:
return_necessary_abstracts(unique_PMIDs, abstracts_output_filename)

# Read in annotations from abstract reading

I read ~150 abstracts to manually classify avian sequences from bird species that can be ambiguous. This was primarily for ducks, geese, turkeys, pigeons, and generic fowl. There were some papers that did not specify where the samples came from, but most did. The vast majority were part of poultry surveillance or outbreak investigation, but there were also a surprising number of wild bird sequences. I ended up doing this in 2 ways: 

1. classifying all sequences for a given paper: for most of the papers, they describe either only domestic or wild, or all of the wild sequences have host species designations that are clearly wild (like "stork"). For these situations, we can just then label all leftover ambiguous sequences from that paper with a single annotation. This is true for the vast majority of abstracts. 

2. classify each strain individually: there were some papers for which they looked at both domestic and wild birds, and the strain names they supply are not really sufficient for classifying (for example, domestic and wild ducks are just labelled "duck"). Luckily, for a few of these papers, they do provide explicit lists in their methods or supplemental materials sections where they specify which strains came from domestic or wild birds. For these situations, I have another text file where I have the strain name, the pubmed id, and the annotation for that specific strain. 

So I will now process these remaining samples as follows: 
1. read in the current metadata with annotations files. For all avian sequences for which `domestic_wild` is currently unknown, and for which `PMID` is not blank, do the following: 
2. first, read in the annotations file specifying a single designation per paper. If the pubmed id was informative, this file will have an annotation of either "domestic" or "wild". If I could not figure it out, the annotation is blank. If while reading the paper I figured out that I needed to annotate each strain separately, this file has an annotation that says "see strain list". 
3. If the annotation is "see strain list", then look up this strain name in the strain names annotation list. 
4. If at the end of all this there still is not an annotation, then we will not be able to annotate it. 

In [19]:
def read_in_classified_abstracts(abstracts_classified_file):
    output_dict = {}
    
    with open(abstracts_classified_file, "r") as infile:
        for line in infile:
            if line != "\n":
                PMID = line.split("\t")[0]
                domestic_wild = line.split("\t")[1]
                notes = line.split("\t")[2]

                output_dict[PMID] = domestic_wild
    return(output_dict)

In [20]:
def read_in_strain_specific_classifications_file(strain_classification_file):
    
    output_dict = {}
    with open(strain_classification_file, "r") as infile: 
        for line in infile:
            if line != "\n":
                strain_name = line.split("\t")[0]                
                domestic_wild = line.split("\t")[1]
                PMID = line.split("\t")[2].strip()
                
                if PMID not in output_dict: 
                    output_dict[PMID] = {}
                
                output_dict[PMID][strain_name] = domestic_wild
    return(output_dict)

In [21]:
def add_manual_annotations(metadata_file, abstracts_dict, strain_abstract_dict, output_filename):
    
    with open(output_filename, "w") as outfile: 
        outfile.write("")
    
    with open(metadata_file, "r") as infile: 
        for line in infile: 
            if "originating_lab" not in line:
                strain = line.split("\t")[0]
                host_group = line.split("\t")[8].lower()
                current_domestic_wild = line.split("\t")[13]
                current_annotation_method = line.split("\t")[14]
                PMID = line.split("\t")[16].strip()
                
                """add in annotations that I manually curated"""
                if host_group == "avian" and current_domestic_wild == "" and PMID != "":
                    if PMID in abstracts_dict and abstracts_dict[PMID] != "":
                        domestic_wild = abstracts_dict[PMID]
                        annotation_method = "manual curation"
                        
                        
                        """if I needed to annotate each strain separately, look that up in the strains file"""
                        if domestic_wild == "see strain list":
                            if PMID in strain_abstract_dict and strain in strain_abstract_dict[PMID]:
                                domestic_wild = strain_abstract_dict[PMID][strain]
                                annotation_method = "manual curation"
                            else:
                                domestic_wild = ""
                    
                    else:
                        domestic_wild = ""
                        annotation_method = ""
                    
                    first_part = "\t".join(line.split("\t")[0:13])
                    last_part = "\t".join(line.split("\t")[15:])
                    line_to_write = "\t".join([first_part,domestic_wild, annotation_method,last_part])
                
                
                # if it already has an annotation, or if it doesn't but there isn't an abstract, move on
                else:
                    line_to_write = line
                
            else:
                line_to_write = line
            
            """write it out"""
            with open(output_filename, "a") as outfile: 
                outfile.write(line_to_write)

In [22]:
abstracts_classified_file = "/Users/lmoncla/src/h5n1-host-classification/abstracts/abstracts-classified-2020-09-28.txt"
strain_classification_file = "/Users/lmoncla/src/h5n1-host-classification/abstracts/strain_specific_annotations.txt"

In [23]:
classified_by_abstract_dict = read_in_classified_abstracts(abstracts_classified_file)
strains_abstract_dict = read_in_strain_specific_classifications_file(strain_classification_file)

In [24]:
metadata_directory = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/"
todays_date = (datetime.datetime.now()).strftime('%Y-%m-%d')
genes_to_run = ['pb2','pb1','pa','ha','np','na','mp','ns']

for gene in genes_to_run: 
    metadata_file = metadata_directory + "metadata_h5n1_" + gene + "_with_accessions_2020-10-02.txt"
    output_filename = metadata_directory + "metadata_h5n1_" + gene + "_final_"+ todays_date+".txt"
    add_manual_annotations(metadata_file, classified_by_abstract_dict, strains_abstract_dict, output_filename)